In [0]:
# File location and type
file_location = "/FileStore/tables/Customer_Segmentation.csv"
file_type = "csv"

df = spark.read.csv(file_location, header= True, inferSchema = True)
df.show()

+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+
|Customer Id|Age|Edu|Years Employed|Income|Card Debt|Other Debt|Defaulted|Address|DebtIncomeRatio|
+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+
|          1| 41|  2|             6|    19|    0.124|     1.073|        0| NBA001|            6.3|
|          2| 47|  1|            26|   100|    4.582|     8.218|        0| NBA021|           12.8|
|          3| 33|  2|            10|    57|    6.111|     5.802|        1| NBA013|           20.9|
|          4| 29|  2|             4|    19|    0.681|     0.516|        0| NBA009|            6.3|
|          5| 47|  1|            31|   253|    9.308|     8.908|        0| NBA008|            7.2|
|          6| 40|  1|            23|    81|    0.998|     7.831|     null| NBA016|           10.9|
|          7| 38|  2|             4|    56|    0.442|     0.454|        0| NBA013|            1.6|
|         

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['Defaulted'],
    outputCols = ['Defaulted_imputed']).setStrategy('median')
df2  = imputer.fit(df).transform(df)
df2.show()
df2.na.drop()
df2.count()
df2.columns
df2.printSchema()

+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+-----------------+
|Customer Id|Age|Edu|Years Employed|Income|Card Debt|Other Debt|Defaulted|Address|DebtIncomeRatio|Defaulted_imputed|
+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+-----------------+
|          1| 41|  2|             6|    19|    0.124|     1.073|        0| NBA001|            6.3|                0|
|          2| 47|  1|            26|   100|    4.582|     8.218|        0| NBA021|           12.8|                0|
|          3| 33|  2|            10|    57|    6.111|     5.802|        1| NBA013|           20.9|                1|
|          4| 29|  2|             4|    19|    0.681|     0.516|        0| NBA009|            6.3|                0|
|          5| 47|  1|            31|   253|    9.308|     8.908|        0| NBA008|            7.2|                0|
|          6| 40|  1|            23|    81|    0.998|     7.831|

In [0]:
from pyspark.ml.feature import VectorAssembler
Assembler = VectorAssembler(inputCols= ['Edu','Income','Years Employed','Card Debt','Other Debt','DebtIncomeRatio','Defaulted_imputed'],
                                outputCol = 'features')
output = Assembler.transform(df2)


+--------------------+
|            features|
+--------------------+
|[2.0,19.0,6.0,0.1...|
|[1.0,100.0,26.0,4...|
|[2.0,57.0,10.0,6....|
|[2.0,19.0,4.0,0.6...|
|[1.0,253.0,31.0,9...|
|[1.0,81.0,23.0,0....|
|[2.0,56.0,4.0,0.4...|
|[3.0,64.0,0.0,0.2...|
|[1.0,18.0,5.0,0.5...|
|[3.0,115.0,23.0,0...|
|[3.0,88.0,8.0,0.2...|
|[2.0,40.0,9.0,0.3...|
|[1.0,18.0,7.0,0.5...|
|[1.0,30.0,6.0,1.4...|
|[3.0,20.0,2.0,0.2...|
|[1.0,16.0,1.0,0.1...|
|[1.0,17.0,1.0,0.1...|
|[4.0,26.0,1.0,1.5...|
|[1.0,61.0,18.0,2....|
|[1.0,32.0,16.0,0....|
+--------------------+
only showing top 20 rows



In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
kmeans = KMeans(featuresCol = 'features')
model = kmeans.fit(output)
prediction = model.transform(output)
prediction.show()

+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+-----------------+--------------------+----------+
|Customer Id|Age|Edu|Years Employed|Income|Card Debt|Other Debt|Defaulted|Address|DebtIncomeRatio|Defaulted_imputed|            features|prediction|
+-----------+---+---+--------------+------+---------+----------+---------+-------+---------------+-----------------+--------------------+----------+
|          1| 41|  2|             6|    19|    0.124|     1.073|        0| NBA001|            6.3|                0|[2.0,19.0,6.0,0.1...|         0|
|          2| 47|  1|            26|   100|    4.582|     8.218|        0| NBA021|           12.8|                0|[1.0,100.0,26.0,4...|         1|
|          3| 33|  2|            10|    57|    6.111|     5.802|        1| NBA013|           20.9|                1|[2.0,57.0,10.0,6....|         0|
|          4| 29|  2|             4|    19|    0.681|     0.516|        0| NBA009|            6.3|        

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-182374488940917> in <module>
      1 evaluator = ClusteringEvaluator(predictionCol = 'prediction',
      2                                featuresCol = 'features')
----> 3 ev = evaluator.transform(prediction)

AttributeError: 'ClusteringEvaluator' object has no attribute 'transform'